In [1]:
#importing required libraries
%matplotlib inline
import os
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import geopy
import geocoder
import requests
from pandas.io.json import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
print('Libraries imported')

Libraries imported


## Scraping pincodes and post offices  in Banglore

In [2]:
CLIENT_ID = '0DWHJ05IBVSSZLFJBNYMJ0VRVZXSDG15UWQWJXMVUL1CRC4X' # your Foursquare ID
CLIENT_SECRET = '3S4WZFTO204JYQX0Z1F4SW02WBZBAETWT5ULBMBA143FDH14' # your Foursquare Secret
VERSION = '20191017' # Foursquare API version
venues=[]
url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        28.725828,  
        77.162787, 
        500, 
        5000)
    
results = requests.get(url).json()["response"]['groups'][0]['items']
index=0
for venue in results:
    index+=1
    venues.append((
        index,
        venue['venue']['name'], 
        venue['venue']['location']['lat'], 
        venue['venue']['location']['lng'],  
        venue['venue']['categories'][0]['name']))
print(venues)

[(1, 'Jahangirpuri Metro Station | जहांगीरपुरी', 28.726025752225787, 77.16262578964233, 'Light Rail Station'), (2, 'Bluebird Pure Private Limited', 28.72523544228677, 77.1623682975769, 'Kitchen Supply Store')]


In [3]:
#scraping makaan.in for housing prices
df=pd.DataFrame(columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties'])
res=pd.read_html('https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi',header=1)
res[0].columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties']
df=df.append(res[0],ignore_index=True)
for n in np.arange(2,59):
    res=pd.read_html('https://www.makaan.com/price-trends/property-rates-for-buy-in-delhi?page={}'.format(n),header=1)
    res[0].columns=['Name','Price range per sqft','Avg price per sqft','Price rise','Trend','View Properties']
    df=df.append(res[0],ignore_index=True)

In [4]:
df

,Name,Price range per sqft,Avg price per sqft,Price rise,Trend,View Properties
0,Uttam Nagar,"843 - 13,333 / sqft","7,486 / sqft",5.4%,See trend,View 468 properties
1,Uttam Nagar west,"996 - 11,548 / sqft","6,762.32 / sqft",-25.7%,See trend,View 98 properties
2,Malviya Nagar,"2,310 - 14,222 / sqft","11,703.7 / sqft",41.1%,See trend,View 12 properties
3,laxmi nagar,"2,353 - 13,889 / sqft","11,858.86 / sqft",-16.3%,See trend,View 588 properties
4,Greater kailash 1,"2,900 - 43,333 / sqft","17,797.51 / sqft",-15%,See trend,View 8 properties
5,Saket,"3,435 - 18,621 / sqft","13,995.16 / sqft",-7%,See trend,View 53 properties
6,Defence Colony,"758 - 35,484 / sqft","20,939.86 / sqft",21.8%,See trend,View 25 properties
7,Safdarjung Enclave,"6,500 - 29,423 / sqft","21,743.02 / sqft",-20.9%,See trend,View 12 properties
8,Vasant Kunj,"2,165 - 1,85,714 / sqft","68,576.87 / sqft",7%,See trend,View 248 properties
9,Greater Kailash II,"3,343 - 18,765 / sqft","16,788.9 / sqft",53.4%,See trend,View 9 properties


In [5]:
#removing unecessary columns price range per sqft, trends,price rise,view properties
df.drop(['Price range per sqft','Price rise','Trend','View Properties'],axis=1,inplace=True)
df

,Name,Avg price per sqft
0,Uttam Nagar,"7,486 / sqft"
1,Uttam Nagar west,"6,762.32 / sqft"
2,Malviya Nagar,"11,703.7 / sqft"
3,laxmi nagar,"11,858.86 / sqft"
4,Greater kailash 1,"17,797.51 / sqft"
5,Saket,"13,995.16 / sqft"
6,Defence Colony,"20,939.86 / sqft"
7,Safdarjung Enclave,"21,743.02 / sqft"
8,Vasant Kunj,"68,576.87 / sqft"
9,Greater Kailash II,"16,788.9 / sqft"


In [6]:
#dropping rows with - as their avg price per sqft
df=df[df['Avg price per sqft']!='-'].reset_index(drop=True)
df

,Name,Avg price per sqft
0,Uttam Nagar,"7,486 / sqft"
1,Uttam Nagar west,"6,762.32 / sqft"
2,Malviya Nagar,"11,703.7 / sqft"
3,laxmi nagar,"11,858.86 / sqft"
4,Greater kailash 1,"17,797.51 / sqft"
5,Saket,"13,995.16 / sqft"
6,Defence Colony,"20,939.86 / sqft"
7,Safdarjung Enclave,"21,743.02 / sqft"
8,Vasant Kunj,"68,576.87 / sqft"
9,Greater Kailash II,"16,788.9 / sqft"


In [7]:
#removing '/sqft' from end of the prices and ',' from values and converting it into float
avgPrice=[]
for price in df['Avg price per sqft'].values:
    price=price.replace(' / sqft','')
    price=price.replace(',','')
    avgPrice.append(price)
df['Avg price per sqft']=avgPrice

In [8]:
#converting Avg price per sqft column into float and renaming columns
df.rename(columns={'Avg price per sqft':'AvgPrice','Name':'AreaName'},inplace=True)
#converting names into lowercase
df['AreaName']=df['AreaName'].str.lower()
df.astype({'AvgPrice': 'float'}).dtypes

AreaName     object
AvgPrice    float64
dtype: object

In [9]:
#removing rows with 'near' in it because it will not lead to an specific location instead vicinity of location
df=df[~df.AreaName.str.contains('near')].reset_index(drop=True)
df

,AreaName,AvgPrice
0,uttam nagar,7486
1,uttam nagar west,6762.32
2,malviya nagar,11703.7
3,laxmi nagar,11858.86
4,greater kailash 1,17797.51
5,saket,13995.16
6,defence colony,20939.86
7,safdarjung enclave,21743.02
8,vasant kunj,68576.87
9,greater kailash ii,16788.9


In [10]:
# removing rows with AreaName containing metro line,highway,expressway,nh,noida,gurgaon,gurugram,peripheral and railway line
df=df[~(df.AreaName.str.contains('metro') | df.AreaName.str.contains('highway')|df.AreaName.str.contains('nh ') | df.AreaName.str.contains('expressway') | df.AreaName.str.contains('noida')| df.AreaName.str.contains('gurgaon')| df.AreaName.str.contains('peripheral')| df.AreaName.str.contains('gurugram')| df.AreaName.str.contains('railway'))].reset_index(drop=True)
df

,AreaName,AvgPrice
0,uttam nagar,7486
1,uttam nagar west,6762.32
2,malviya nagar,11703.7
3,laxmi nagar,11858.86
4,greater kailash 1,17797.51
5,saket,13995.16
6,defence colony,20939.86
7,safdarjung enclave,21743.02
8,vasant kunj,68576.87
9,greater kailash ii,16788.9


In [11]:
#exporting dataframe to csv
# df.to_csv('pricing.csv')

In [12]:
#generating authorization header for accessing mapmyindia geocoding API
res=requests.post('https://outpost.mapmyindia.com/api/security/oauth/token',\
                 params={'grant_type': 'client_credentials',
                        'client_id':'DQMqcA9v_ZoFp2hAoT0VIH6U88jDzGjmoeBUK1HnzhF0mWj-m8pb9z6ad-i-DlT52EY2o3TFV-1bVGjavkTIkA==',
                        'client_secret':'ebEc8GH231ciXN2DqeNUYsS-WJMaqjPeL2MrXBJEkASOSsLto-LBnhAxbZhCITzGlBFkJ9-vrMkuVdFfXf06pIBKp5WJI3Z9'},\
                 headers={'Content-Type': 'application/x-www-form- urlencoded'})
map_my_india_token=res.json()['access_token']

In [13]:
#using mapmyindia geocoding api to get latitude and longitude of areas in dataframe
latitudes=[]
longitudes=[]
pinCodes=[]
districts=[]
for area in df['AreaName'].values:
    res=requests.get('https://atlas.mapmyindia.com/api/places/geocode?address={},Delhi'.format(area),\
                headers={'Authorization':'{}'.format(map_my_india_token)})
    latitudes.append(res.json()['copResults']['latitude'])
    longitudes.append(res.json()['copResults']['longitude'])
    pinCodes.append(res.json()['copResults']['pincode'])
    districts.append(res.json()['copResults']['district'])

In [14]:
# merging latitudes and longitudes into new columns
df['Latitude']=pd.Series(latitudes)
df['Longitude']=pd.Series(longitudes)
df['PinCode']=pd.Series(pinCodes)
df['District']=pd.Series(districts)
df

,AreaName,AvgPrice,Latitude,Longitude,PinCode,District
0,uttam nagar,7486,28.618535,77.056782,110059,West District
1,uttam nagar west,6762.32,28.618535,77.056782,110059,West District
2,malviya nagar,11703.7,28.534883,77.210245,110017,South District
3,laxmi nagar,11858.86,28.635202,77.283208,110092,East District
4,greater kailash 1,17797.51,28.548714,77.236102,110048,South East Delhi District
5,saket,13995.16,28.523848,77.206773,110017,South District
6,defence colony,20939.86,28.573199,77.232795,110024,South East Delhi District
7,safdarjung enclave,21743.02,28.565741,77.194877,110029,South District
8,vasant kunj,68576.87,28.514980,77.153245,110070,South District
9,greater kailash ii,16788.9,28.534883,77.241845,110048,South East Delhi District


In [15]:
# Checking if there is any null value in whole dataframe
if(df.isnull().values.all()!=True):
    print('No null values present!!')

No null values present!!


In [19]:
#capitalizing area names
areaNames=[]
for area in df['AreaName'].values:
    area=area.title()
    areaNames.append(area)
df['AreaName']=areaNames
df.head()

,AreaName,AvgPrice,Latitude,Longitude,PinCode,District
0,Uttam Nagar,7486,28.618535,77.056782,110059,West District
1,Uttam Nagar West,6762.32,28.618535,77.056782,110059,West District
2,Malviya Nagar,11703.7,28.534883,77.210245,110017,South District
3,Laxmi Nagar,11858.86,28.635202,77.283208,110092,East District
4,Greater Kailash 1,17797.51,28.548714,77.236102,110048,South East Delhi District


#### Creating map of Delhi with Areas on top of it

In [20]:
map_delhi=folium.Map(location=[28.644800, 77.216721],zoom_start=11)

#adding markers to map
for lat,lng,area,pin,dist in zip(df['Latitude'],df['Longitude'],df['AreaName'],df['PinCode'],df['District']) :
    label='{},{}-{}'.format(area,dist,pin)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7).add_to(map_delhi)

map_delhi

In [21]:
#exporting this dataframe
df.to_csv('area_lat_long.csv')